In [1]:
import pandas as pd
import numpy as np 
data = pd.read_csv('training_data.csv')  ##use this data cv F1 0.50  
#data = pd.read_csv('facies_vectors.csv')  ##use this data(two more wells) cv F1 0.48 
#data.describe()
#test_well = data[data['Well Name'] == 'SHANKLE']
#data = data[data['Well Name'] != 'SHANKLE']
#data2 = data.sample(n=729)
#data2.describe()   # use random sampled 729 events gives 
#F1 scroe 0.51 for CV(nonstratified split), 0.36 for Test 
#F1 scroe 0.55 for CV(stratified split), 0.41 for Test 

#data['BoundaryFlag'] = pd.Series(0,index=data.index)  # add one column. 
#previousface=-1
#for index, row in data.iterrows():
#    if row['Facies']!=previousface: 
#        data.at[index, 'BoundaryFlag'] =  1   
#        data.at[index-1, 'BoundaryFlag'] =  1    
#        data.set_value(index, 'BoundaryFlag',1) ##old version
#        previousface = row['Facies']
#data = data[data['BoundaryFlag']==0]
#data= data.drop(['BoundaryFlag'],axis=1)
#data.describe
#w/o boundary in data, F1 score 0.61 for CV and 0.43 for Test 
#w boundary in data, F1 score 0.50 for CV and 0.55 for Test
data.index

RangeIndex(start=0, stop=3232, step=1)

In [2]:
features = ['GR', 'ILD_log10', 'DeltaPHI','PHIND','PE','NM_M', 'RELPOS']
feature_vectors = data[features]
facies_labels = data['Facies']
facies_labels.describe()
# Store well labels and depths
well = data['Well Name'].values
depth = data['Depth'].values

# Fill 'PE' missing values with mean
from sklearn import preprocessing
imp = preprocessing.Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(feature_vectors)
feature_vectors = imp.transform(feature_vectors)

In [3]:



# Feature windows concatenation function
def augment_features_window(X, N_neig):
    
    # Parameters
    N_row = X.shape[0]
    N_feat = X.shape[1]

    # Zero padding
    X = np.vstack((np.zeros((N_neig, N_feat)), X, (np.zeros((N_neig, N_feat)))))

    # Loop over windows
    X_aug = np.zeros((N_row, N_feat*(2*N_neig+1)))
    for r in np.arange(N_row)+N_neig:
        this_row = []
        for c in np.arange(-N_neig,N_neig+1):
            this_row = np.hstack((this_row, X[r+c]))
        X_aug[r-N_neig] = this_row

    return X_aug


# Feature gradient computation function
def augment_features_gradient(X, depth):
    
    # Compute features gradient
    d_diff = np.diff(depth).reshape((-1, 1))
    d_diff[d_diff==0] = 0.001
    X_diff = np.diff(X, axis=0)
    X_grad = X_diff / d_diff
        
    # Compensate for last missing value
    X_grad = np.concatenate((X_grad, np.zeros((1, X_grad.shape[1]))))
    
    return X_grad


# Feature augmentation function
def augment_features(X, well, depth, N_neig=1):
    
    # Augment features
    X_aug = np.zeros((X.shape[0], X.shape[1]*(N_neig*2+2)))
    for w in np.unique(well):
        w_idx = np.where(well == w)[0]
        X_aug_win = augment_features_window(X[w_idx, :], N_neig)
        X_aug_grad = augment_features_gradient(X[w_idx, :], depth[w_idx])
        X_aug[w_idx, :] = np.concatenate((X_aug_win, X_aug_grad), axis=1)
    
    # Find padded rows
    padded_rows = np.unique(np.where(X_aug[:, 0:7] == np.zeros((1, 7)))[0])
    
    return X_aug, padded_rows

In [15]:
X_aug, padded_rows = augment_features(feature_vectors, well, depth)
feature_vectors.shape

(3232, 7)

In [16]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_aug)
scaled_features = scaler.transform(X_aug) #ndarray now. 
scaled_features.shape
#df3 = pd.DataFrame(np.random.randn(10, 5),columns=['a', 'b', 'c', 'd', 'e'])
#df3.describe
X_aug.shape

(3232, 28)

In [19]:
#test_well = X_aug[X_aug['Well Name'] == 'SHANKLE']
#data = X_aug[X_aug['Well Name'] != 'SHANKLE']
df = pd.DataFrame(data=scaled_features)
df.describe

<bound method NDFrame.describe of             0         1         2         3         4         5         6   \
0    -2.129533 -2.631777 -0.679258 -1.744977 -4.065415 -2.958108 -1.808912   
1     0.368629  0.096338  1.213540 -0.201197  0.969265 -0.977456  1.672811   
2     0.394756  0.084012  2.035664 -0.116979  0.422017 -0.977456  1.599694   
3     0.420237  0.071686  2.150379 -0.054139 -0.125231 -0.977456  1.523097   
4     0.647636  0.059360  1.978306 -0.045717 -0.234681 -0.977456  1.449980   
5     0.276057  0.026492  1.901830 -0.021748 -0.344130 -0.977456  1.376864   
6     0.256381 -0.018703  1.997426 -0.010734 -0.125231 -0.977456  1.303748   
7     0.248317 -0.043355  2.303332  0.059879 -0.015781 -0.977456  1.227150   
8     0.310570 -0.063898  2.475405  0.058583 -0.234681 -0.977456  1.080918   
9     0.250575 -0.068006  2.418047  0.066357 -0.344130 -0.977456  1.007802   
10    0.350566 -0.104984  2.551881  0.097453 -0.234681 -0.977456  0.931204   
11    0.325407 -0.166613  2.15

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(scaled_features, facies_labels,test_size=0.05, random_state=42,stratify=facies_labels)
#X_train, X_cv, y_train, y_cv = train_test_split(scaled_features, facies_labels,test_size=0.05, random_state=42)
#y_train.value_counts()
y_cv.value_counts()

In [ ]:
from sklearn.metrics import classification_report
target_names = ['SS', 'CSiS', 'FSiS', 'SiSh','MS', 'WS', 'D','PS', 'BS']

In [ ]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-1,hidden_layer_sizes=(3), random_state=1)
clf.fit(X_train, y_train)

In [ ]:
y_cv_pred = clf.predict(X_cv) 
print(classification_report(y_cv, y_cv_pred,target_names=target_names))

In [ ]:
y_test = test_well['Facies']

In [ ]:
well_features = test_well.drop(['Facies','Formation','Well Name','Depth'],axis=1)
X_test = scaler.transform(well_features)
y_pred = clf.predict(X_test)
test_well['Prediction'] = y_pred

In [ ]:

print(classification_report(y_test, y_pred,target_names=target_names))